<a href="https://colab.research.google.com/github/MMarck/PLN_Desafio_1_Vectorizacion/blob/main/Desafio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [4]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [5]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [6]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [7]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [8]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [9]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [10]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [11]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [12]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:7]

In [13]:
idx = 4546
print(newsgroups_train.data[idx])

Can someone recommend how to ship a motorcycle from San Francisco
to Seattle?  And how much might it cost?

I remember a thread on shipping.  If someone saved the instructions
on bike prep, please post 'em again, or email.

Thanks,


In [14]:
# midamos la similaridad coseno con todos los documentos de train
cossim_2 = cosine_similarity(X_train[idx], X_train)[0]
most_sim_2 = np.argsort(cossim_2)[::-1][0:7]

In [15]:
np.sort(cossim_2)[::-1][0:7]

array([1.        , 0.20928553, 0.2008908 , 0.1665626 , 0.1624009 ,
       0.1582626 , 0.15815911])

In [16]:
# visualizacion del valor de similaridad
np.sort(cossim_2)[::-1][1:6]

array([0.20928553, 0.2008908 , 0.1665626 , 0.1624009 , 0.1582626 ])

In [17]:
np.sort(cossim_2)[::-1][0:7]

array([1.        , 0.20928553, 0.2008908 , 0.1665626 , 0.1624009 ,
       0.1582626 , 0.15815911])

In [18]:
print(newsgroups_train.data[most_sim_2[1]])

Could someone out there please tell me how I could get onto
the Saab mailing list. Specifically I need the address and
instructions on what to do.

Thanks in advance


In [19]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'rec.motorcycles'

In [20]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


## Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


## 1. Análisis de documento aleatorios

In [21]:
def most_similar(indice, doc_vectorizados, target_arr, imprimir_doc_sim):
  """
  Muestra los documentos más similares a un documento dado utilizando la similitud coseno.

  Este método imprime el contenido del documento de análisis (según el índice dado) y luego
  muestra los 5 documentos más similares de un conjunto de documentos vectorizados.
  La similitud se mide mediante la similitud coseno.

  Args:
      indice (int): Índice del documento que se va a analizar.
      doc_vectorizados (array-like): Matriz de documentos vectorizados (solo usando TF-IDF).
      imprimir_doc_sim (bool): Si es True, imprime el contenido completo de los documentos similares;
                                si es False, solo muestra los índices y la puntuación de similitud.

  Returns:
      list[int]: Lista con los índices de los documentos más similares (ordenados de mayor a menor similitud).
  """
  print("=========== Documento de análisis ===========")
  print(f"----- indice: {indice} ------- etiqueta: {newsgroups_train.target_names[target_arr[indice]]}---------")
  print(newsgroups_train.data[indice])
  print("=============================================")
  print("\n\n")

  # midamos la similaridad coseno con todos los documentos de train
  cosine_sim_arr = cosine_similarity(doc_vectorizados[indice], doc_vectorizados)[0]
  most_sim_indices = np.sort(cosine_sim_arr)[::-1][0:6]
  most_sim_arr = np.argsort(cosine_sim_arr)[::-1][0:6]

  if (imprimir_doc_sim):
    for i in range(len(most_sim_indices)):
      print(f"-------------- documento con indice:{most_sim_arr[i]} ----------------------------")
      print(f"-------------- similaridad coseno: {most_sim_indices[i]:.4f} ----------------------------")
      print(f"-------------- etiqueta: {newsgroups_train.target_names[target_arr[i]]} ----------------------------")

      print(newsgroups_train.data[most_sim_arr[i]])
      print(f"-----------------------------------------------------------------------------------")
      print("\n\n")
  else:
    for i in range(len(most_sim_indices)):
      print(f"documento con indice:{most_sim_arr[i]} - similaridad coseno: {most_sim_indices[i]:.4f} - etiqueta: {newsgroups_train.target_names[target_arr[i]]}")

  return most_sim_arr

In [22]:
most_similar(4811, X_train, y_train, False)

=========== Documento de análisis ===========
----- indice: 4811 ------- etiqueta: talk.politics.misc---------
THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
        

array([4811, 6635, 4253, 3596, 4271, 3746])

### **Análisis documento con indice 4811**
Es el dialogo de una emisora de radio que realiza un
entrevista al presidente de EEUU. Le realizan varias preguntas,
mientas el presidente responde sobre recortes y desempleo.

* El documento con indice **6635** es un discurso del presidente Clinton donde destacó la importancia de brindar oportunidades laborales y educativas a los jóvenes estadounidenses. Anunció un programa de empleos de verano con fondos publicos y privados. El programa busca reformar el bienestar social, Capacitar trabajadores desempleados y Fortalecer la economía local con inversiones en infraestructura.
* El documento con indice **4253** El texto es la transcripcion del evento publico Town Hall por satelite donde el presidente Clinton hablo de las metas para el anio 200. Entre ellas: cómo preparar mejor a los jóvenes con una transición efectiva de la escuela al empleo. Tambien se hablo de la educación como base de la democracia. Hablo de la importancia de fomentar el aprendizaje continuo. Presento un programa de empleos de verano con componente educativo
* El documento con indice **3596** La Casa Blanca anunció un paquete de ayuda de 1.6 mil millones para apoyar las reformas económicas y políticas en Rusia, acordado entre los presidentes Clinton y Yeltsin. Este paquete, financiado con fondos ya aprobados por el Congreso, incluye asistencia humanitaria, préstamos alimentarios concesionales, apoyo al sector privado, democratización, reasentamiento de oficiales militares rusos, cooperación en energía y medio ambiente, y el desmantelamiento seguro de arsenales nucleares. También se enfatiza la importancia del comercio e inversión
como motor del desarrollo ruso, incluyendo créditos para proyectos energéticos
y apoyo a la integración de Rusia en la economía global. Todo está diseñado para ejecutarse de inmediato, sin necesidad de nueva aprobación legislativa.
* El documento con indice **4271** En esta conferencia de prensa conjunta con la prensa rusa, el presidente Clinton reafirma que el paquete de ayuda a Rusia no es una dádiva, sino una inversión estratégica en la democracia, la estabilidad y el desarrollo económico, que también beneficia a Estados Unidos al reducir amenazas como la guerra y las armas nucleares. Clinton subraya que el dinero será distribuido principalmente en iniciativas privadas y que se busca incluir a empresarios rusos en la planificación. También destacó su interés personal por la cultura rusa y su compromiso con una relación cercana entre ambos países. Finalmente, abordó temas como la ratificación de los tratados START y la posible eliminación de restricciones heredadas de la
Guerra Fría, como Jackson-Vanik y COCOM.
* El documento con indice **3746** Este documento detalla un informe técnico de alto nivel del gobierno estadounidense sobre un paquete de ayuda de $1.6 mil millones destinado a respaldar las reformas económicas y políticas en Rusia tras el colapso de la Unión Soviética. Las iniciativas incluyen asistencia alimentaria y médica, préstamos agrícolas con condiciones concesionales, apoyo a la privatización y desarrollo de pequeñas empresas, proyectos de democratización, reasentamiento de oficiales militares, cooperación en energía y medioambiente, y desarme nuclear seguro. Todo el paquete está financiado con fondos ya aprobados para el año fiscal 1993, sin necesidad de nueva autorización del Congreso. Además, se enfatiza el papel del sector privado y del comercio como pilares sostenibles del desarrollo ruso, con mecanismos de supervisión para asegurar la correcta aplicación de los recursos. También se destaca la intención de Estados Unidos de liderar este esfuerzo junto a sus aliados occidentales, buscando no solo estabilidad en Rusia, sino beneficios estratégicos mutuos en términos de seguridad, comercio y cooperación global.

### **Conclusión**
Los documentoss mantienen relacion entre ellos, notando que hay terminos ern comun como: presidente, clinton, OFFICE. Estos terminos se repiden en todos y permiten a los vectores guardar una relacion.






In [23]:
most_similar(4546, X_train, y_train, True)

=========== Documento de análisis ===========
----- indice: 4546 ------- etiqueta: rec.motorcycles---------
Can someone recommend how to ship a motorcycle from San Francisco
to Seattle?  And how much might it cost?

I remember a thread on shipping.  If someone saved the instructions
on bike prep, please post 'em again, or email.

Thanks,



-------------- documento con indice:4546 ----------------------------
-------------- similaridad coseno: 1.0000 ----------------------------
-------------- etiqueta: rec.autos ----------------------------
Can someone recommend how to ship a motorcycle from San Francisco
to Seattle?  And how much might it cost?

I remember a thread on shipping.  If someone saved the instructions
on bike prep, please post 'em again, or email.

Thanks,
-----------------------------------------------------------------------------------



-------------- documento con indice:8494 ----------------------------
-------------- similaridad coseno: 0.2093 ---------------------

array([4546, 8494, 9823, 4489, 9803, 3269])

### **Análisis documento con indice 4546**
Habla acerca de una consulta que alguien realizo acerca de como enviar una motocicleta por correo

* El documento con indice **8494** es una consulta donde se pregunta por el email de una tal Saab. Se observa que hay terminos que se comparten como, instructions y thanks
* el documento con indice **9823** es un consejo que alguien comparte acerca de como comprar un motocicleta facil de asegurar. Se observa que hay algunos terminos que comparten como bike,
* El documento con indice **4489** es una respuesta. donde se agradece por alguna informacion proporcionada. Se observa que comparte el termino cost
* El documento con indice **9803** parece un correo con instrucciones para usar un servicio en el que se puede enviar motocicletas cierto medio. Se observan terminos compartidos como cost.
* El documento con indice **3269** es un correo que cuenta una experiencia sobre como alguien envio su motocileta. Se observan algunos terminos en comun como bike.

### **Conclusión**
Los documentos guardan una relacion. Se observa claramente como todos hablan acerca de motocicletas, mientras que alguine hablan acerca de como comprar una otros hablan sobre como enviarla.

In [24]:
most_similar(1624, X_train, y_train, True)

=========== Documento de análisis ===========
----- indice: 1624 ------- etiqueta: misc.forsale---------
I have the following games for sale or trade for other SNES (or
Genesis/MegaDrive games):
(all have instructions and box except where stated)

SFC:
Mickeys Magical Quest (no instructions)
A.Suzukis Super GrandPrix
Legend of the Mystical Ninja

UK SNES:
Out of this World / Another World
Super Soccer

US SNES:
Krustys Fun House
Irem Skins Golf
Super Tennis (currently under offer)

I will sell for US$ for UK pounds.

Cheers
Jonathan




-------------- documento con indice:1624 ----------------------------
-------------- similaridad coseno: 1.0000 ----------------------------
-------------- etiqueta: rec.autos ----------------------------
I have the following games for sale or trade for other SNES (or
Genesis/MegaDrive games):
(all have instructions and box except where stated)

SFC:
Mickeys Magical Quest (no instructions)
A.Suzukis Super GrandPrix
Legend of the Mystical Ninja

UK SNES:

array([ 1624,  4896,   525, 10109,  5061,  9382])

### **Análisis documento con indice 1624**
Es un anuncio donde se comenta que se dispones de juegos de SNES para venta o intercambio.

* El documento con indice **4896** tambien es un anuncion para vender o intercambiar juegos. Se obserba que usan la misma frase "sale or trade", ademas del "Cheers" al final de ambos documentos
* El documento con indice **525** Es un anuncio de un dispositivo para clonar juegos, realizar guardado de partidas, modificar la reproduccion del juego, entre otras funciones. Se observa que el termino Genesis y game es comun en mabos documentos. Tambien resaltar que a pesar de tener un coeficiente de similaridad bastante bajo y que los documentos tienen diferentes equetas, se mantiene una relacion tematica, ya que ambos documentos hablan acerca de videojuegos
* El documento con indice **10109** es tambien un anuncio para vender o intercambiar juegos. Los terminos similares en este caso son genesis y games. Este documento tambien tiene una etiqueta diferente al documento de analisis
* El documento con indice **5061** Este no es un anuncio de compra venta de videojuegos, sino una consulta por una tarjeta de video para un computador IBM, la persona menciona copiadores de SNES y Genesis como opcion, por lo que la mencion de estos terminos le permitio al algoritmo identificar una relacion.
* El documento con indice **9382** es un anuncio para la venta de juego de nintendo. este documento guarda mayor relacion senantica con el documento de analisis aunque no se usan tantos terminos en comun.

### **Conclusión**
Todos los documentos mantienen una relacion semantica salvo un con el que no se guarda tanta relacion pero si se mencionan elementos en comun. En todos ellos el elemento en comun es Genesis o SNES.

In [25]:
most_similar(4445, X_train, y_train, True)

=========== Documento de análisis ===========
----- indice: 4445 ------- etiqueta: rec.autos---------
Are there any Honda groups out there?  Especially ones that deal with
Preludes?



-------------- documento con indice:4445 ----------------------------
-------------- similaridad coseno: 1.0000 ----------------------------
-------------- etiqueta: rec.autos ----------------------------
Are there any Honda groups out there?  Especially ones that deal with
Preludes?
-----------------------------------------------------------------------------------



-------------- documento con indice:6228 ----------------------------
-------------- similaridad coseno: 0.2291 ----------------------------
-------------- etiqueta: comp.sys.mac.hardware ----------------------------

It's there.....
-----------------------------------------------------------------------------------



-------------- documento con indice:6175 ----------------------------
-------------- similaridad coseno: 0.2048 ----------

array([4445, 6228, 6175, 2923, 1525, 2529])

### **Análisis documento con indice 4445**
Este documento es una pregunta corta donde se consulta por un grupo sobre Honda y hace enfasis en los relacionados con el modelo de automovil Prelude de Honda.


* El documento con indice **6228** es una respuesta corta "It's there.....". En esta ocasion la relacion que guardan estos documento es el uso del termino "there" al principio de ambos, pero al ser este un documento tan corto no tiene un trasfondo con el que puedan relacionar. Ademas las etiquetas de ambos documentos son distintas
* El documento con indice **6175** Es un dato informativo sobre el un modelo de auto Honda. En mabos documentos se usa el termino HONDA.
* El documento con indice **2923** Es una respuesta a lo que parece ser un error en un sistema operativo. Este documento no solo esta etiquetado con un valor diferente al documento de analisis sino que no guarda una relacion semantica con el documento.
* El documento con indice **1525** parece ser un anuncio que promocia el programa Norton Desktop. la etiqueta es diferente al documento de analisis, ademas este documento guarda mas relacion semantica con el documento con indice 2923 que con el documento de analisis
* El documento con indice **2529** es una consulta sobre sitios para intercambiar archivos. tampoco guarada relacion semantica con el documento de analisis. A pesar de que en este documento hay terminos en comun con el documento de analisis como "there" este documento tiene un menor coeficiente de similaridad. Posiblemente el menor coeficiente de similaridad se deba a que este documento es mas extenso que los anteriores.

### **Conclusión**
Los documentos con una longitud baja tiene una dificultad para relacionarse con otros documentos. Los coeficientes de similaridad de todos los documentos fueron menores a 0.3 y las relaciones semanticas entre la mayoria de documentos no es muy alta y en algunos casos nula.




In [26]:
most_similar(780, X_train, y_train, True)

=========== Documento de análisis ===========
----- indice: 780 ------- etiqueta: rec.sport.baseball---------

too bad he doesn't bring the ability to hit, pitch, field or run.



-------------- documento con indice:780 ----------------------------
-------------- similaridad coseno: 1.0000 ----------------------------
-------------- etiqueta: rec.autos ----------------------------

too bad he doesn't bring the ability to hit, pitch, field or run.
-----------------------------------------------------------------------------------



-------------- documento con indice:6690 ----------------------------
-------------- similaridad coseno: 0.2843 ----------------------------
-------------- etiqueta: comp.sys.mac.hardware ----------------------------

I remember one he hit circa 1976 at Wrigley Field that went across
the street (in dead center field) and hit a house on the roof.  He
whiffed a lot, but when he *did* connect, watch out!




-- 

#include <std_disclaimer.h>
--------------------

array([ 780, 6690, 4226, 4461, 1316, 3318])

### **Análisis documento con indice 780**
El documento parce ser un titlo que critica la habilidad de una persona para realizar movimientos de baseball. La etiqueta de este texto no parece guardar relacion con el documento.

* El documento con indice **6690** Este documento es una narracion de una jugada de baseball, comparte terminos en comun con el documento de analisis. La etiqueta del texto y el pie de pagina indican que el documento enrealidad es un error de salida, pero se tomara la primera porcion del texto para el analisis. Aunque el documento habla de forma positiva sobre la jugada, contrario al documento de analisis, este guarda una relacion semantica alta.
* El documento con indice **4226** es una nararacion de las mejores jugadas de un basebolista. El documento presenta terminos en comun con el documento de analisis como "Hit" o "run". a pesar de la corta longitud del documento de analisis ambos documento mantienen cierta relacion ya que ambos hablan a cerca del baseball
* El documento con indice **4461** es un breve reporte informal sobre una lesión sufrida por un jugador (Dean), quien fue golpeado por un lanzamiento en el codo. El texto proporciona detalles sobre la lesión, su evaluación médica (radiografías) y la expectativa de recuperación.
* El documento con indice **1316** es una narración nostálgica y positiva que resalta sus capacidades como bateador, en especial sus famosos jonrones. Describe tanto los "moon shots" (batazos altísimos y largos) como los "microwave home runs" (jonrones rápidos y potentes), incluyendo una anécdota detallada de un juego de 1988. El tono es elogioso y entusiasta.
* El documento con indice **3318** es un breve informe informativo sobre un lanzador que fue retirado de un juego debido a molestias físicas. La causa se atribuye al clima frío en Denver y al intento fallido de soltar el brazo tras lanzar una curva. Se indica que no se trata de una lesión grave y que se espera que lance en su próxima aparición programada.

### **Conclusión**
A pesar de la corta longitud del documento de analisis los documentos obtuvieron valores de coeficientes de similaridad menores 0.3. Sin embargo todos mantenian una relacion semantica ligera ya que todos hablaban de alguna forma sobre baseball. Posiblemente el uso de terminologias del deporte permitio mantener una relacion entre los documentos.

## 2. Modelo de clasificación Naïve Bayes

### MultinomialNB

In [39]:
!wandb login 3b049773b9eb61e83a004b84e1e119cbd7969746

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [41]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

# Definir el clasificador base
nb = MultinomialNB()

# Definir la grilla de hiperparámetros a evaluar
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0],
    'fit_prior': [True, False]
}

# Crear el GridSearchCV
grid_search = GridSearchCV(estimator=nb, param_grid=param_grid, cv=5, scoring='f1_macro')

# Ajustar a los datos
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros
print("Mejores hiperparámetros:", grid_search.best_params_)

# Usar el mejor modelo
best_model = grid_search.best_estimator_


Mejores hiperparámetros: {'alpha': 0.1, 'fit_prior': False}


In [64]:
import wandb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

# Inicializa wandb con un nombre de proyecto
wandb.init(project="pnl_desafio_1", name="run-tfidf-mnb")

# Define hiperparámetros
config = wandb.config
config.min_df = 3
config.stop_words = 'english'
config.norm = 'l2'
config.sublinear_tf = False
config.alpha = 0.1
config.fit_prior = False

# Vectorización
tfidfvect_2 = TfidfVectorizer(
    min_df=config.min_df,
    stop_words=config.stop_words,
    norm=config.norm,
    sublinear_tf=config.sublinear_tf
)
X_train = tfidfvect_2.fit_transform(newsgroups_train.data)

# Modelo
clf = MultinomialNB(alpha=config.alpha, fit_prior=config.fit_prior)
clf.fit(X_train, y_train)

# Predicciones
X_test = tfidfvect_2.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred = clf.predict(X_test)

# Evaluación
f1 = f1_score(y_test, y_pred, average='macro')

# Registro de métricas
wandb.log({"f1_macro": f1})


f1_macro,▁
f1_macro,0.68847


### ComplementNB


In [75]:
import wandb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

# Inicializa wandb con un nombre de proyecto
wandb.init(project="pnl_desafio_1_complement", name="run-tfidf-mnb")

# Define hiperparámetros
config = wandb.config
config.max_df = 0.95
config.min_df = 3
config.norm = 'l2'
config.sublinear_tf = True

# Vectorización
tfidfvect_2 = TfidfVectorizer(
    max_df=config.max_df,
    min_df=config.min_df,
    norm=config.norm,
    sublinear_tf=config.sublinear_tf
)
X_train = tfidfvect_2.fit_transform(newsgroups_train.data)

# Modelo
clf = ComplementNB()
clf.fit(X_train, y_train)

# Predicciones
X_test = tfidfvect_2.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred = clf.predict(X_test)

# Evaluación
f1 = f1_score(y_test, y_pred, average='macro')

# Registro de métricas
wandb.log({"f1_macro": f1})


f1_macro,▁
f1_macro,0.6907


## 3. Terminos similares

In [34]:
def most_similar_word(termino, index2word_voc, doc_vectorizados, target_arr, imprimir_doc_sim):
  """
  Muestra los documentos más similares a un documento dado utilizando la similitud coseno.

  Este método imprime el contenido del documento de análisis (según el índice dado) y luego
  muestra los 5 documentos más similares de un conjunto de documentos vectorizados.
  La similitud se mide mediante la similitud coseno.

  Args:
      indice (int): Índice del documento que se va a analizar.
      doc_vectorizados (array-like): Matriz de documentos vectorizados (solo usando TF-IDF).
      imprimir_doc_sim (bool): Si es True, imprime el contenido completo de los documentos similares;
                                si es False, solo muestra los índices y la puntuación de similitud.

  Returns:
      list[int]: Lista con los índices de los documentos más similares (ordenados de mayor a menor similitud).
  """
  indice = tfidfvect.vocabulary_[termino]
  print("")
  print(f"=========== Termino de análisis: {termino} - Indice: {indice} ===========")
  print("")

  # midamos la similaridad coseno con todos los documentos de train
  cosine_sim_arr = cosine_similarity(doc_vectorizados[indice], doc_vectorizados)[0]
  most_sim_indices = np.sort(cosine_sim_arr)[::-1][0:6]
  most_sim_arr = np.argsort(cosine_sim_arr)[::-1][0:6]

  if (imprimir_doc_sim):
    for i in range(len(most_sim_indices)):
      print(f"Termino: {index2word_voc[most_sim_arr[i]]} - Indice:{most_sim_arr[i]} - Similaridad coseno: {most_sim_indices[i]:.4f}")
  else:
    for i in range(len(most_sim_indices)):
      print(f"Termino con indice:{most_sim_arr[i]} - similaridad coseno: {most_sim_indices[i]:.4f}")

  return most_sim_arr

In [35]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)
X_train_transposed = X_train.transpose()
y_train = newsgroups_train.target
y_train[:10]


array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [36]:
# cossim = cosine_similarity(X_train[idx], X_train)[0]
X_train_transposed[0]

<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 243 stored elements and shape (1, 11314)>

In [37]:
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [38]:
most_similar_word('hit', idx2word, X_train_transposed, y_train, True)
most_similar_word('game', idx2word, X_train_transposed, y_train, True)
most_similar_word('thanks', idx2word, X_train_transposed, y_train, True)
most_similar_word('car', idx2word, X_train_transposed, y_train, True)
most_similar_word('card', idx2word, X_train_transposed, y_train, True)



=========== Termino de análisis: hit - Indice: 46816 ===========

Termino: hit - Indice:46816 - Similaridad coseno: 1.0000
Termino: whiffed - Indice:96456 - Similaridad coseno: 0.2402
Termino: pinch - Indice:71472 - Similaridad coseno: 0.2142
Termino: field - Indice:40224 - Similaridad coseno: 0.2142
Termino: richmond - Indice:78331 - Similaridad coseno: 0.2065
Termino: klesko - Indice:54520 - Similaridad coseno: 0.2057

=========== Termino de análisis: game - Indice: 42595 ===========

Termino: game - Indice:42595 - Similaridad coseno: 1.0000
Termino: games - Indice:42606 - Similaridad coseno: 0.2116
Termino: espn - Indice:37954 - Similaridad coseno: 0.1935
Termino: hockey - Indice:47021 - Similaridad coseno: 0.1806
Termino: team - Indice:87968 - Similaridad coseno: 0.1786
Termino: scored - Indice:81000 - Similaridad coseno: 0.1707

=========== Termino de análisis: thanks - Indice: 88509 ===========

Termino: thanks - Indice:88509 - Similaridad coseno: 1.0000
Termino: advance - Indic

array([25812, 94303, 20191, 25827, 94220, 79911])

### conclusion del punto 3

Se puede obtener terminos con cierto grafdo de relacion. en ninguno de los ejemplos que se probo el valor de la similaridad supero el 0.3. Sin embargo los terminos presentaba cierta relacion entre ellos, esto posiblemente porque el aun guarden cierta corcodancia al ser usados de la misma manera a los largo de los documentos.